# Climate forcing correction
## Juncal Norte - august, 2022
#### Paul Sandoval Quilodrán - https://github.com/SQPaul

#### Import packages

In [1]:
import glob
import os
from osgeo import gdal
import pandas as pd
import rasterio as rio
from rasterio.plot import show
import numpy as np
import xarray as xr
import cftime

#### Calculate Juncal_en_Juncal station cell position 

In [4]:
stations_path = r"C:\Users\pauls\Anaconda3\pkgs\sphy-2.2.1-py_0\site-packages\SPHY\Juncal_norte\input\outflow500.map"
stations = rio.open(stations_path).read(1)

for i in range(stations.shape[0]):
    for j in range(stations.shape[1]):
        #print(i,j)
        if stations[i,j] == 2:
            x = i
            y = j
print(x,y)

36 21


#### Create net_dem (altitude Juncal_en_Juncal station = 0)

In [ ]:
#Read dem
dem_tif = rio.open(r"C:\Users\pauls\Desktop\Proyectos\Juncal_norte\GIS\dem500.tif")
dem = dem_tif.read(1)

for i in range(dem.shape[0]):
    for j in range(dem.shape[1]):
        dem[i,j] = int(dem[i,j])-2250
        
net_dem = rio.open(r"C:\Users\pauls\Desktop\Proyectos\Juncal_norte\GIS\net_dem.tif",
                   "w",
                   driver = "GTiff",
                   height = dem.shape[0],
                   width = dem.shape[1],
                   count = 1,
                   nodata = np.nan,
                   dtype = np.float32,
                   crs = 32719,
                   transform = dem_tif.transform)

net_dem.write(dem,1)
net_dem.close()

### Precipitation

#### Generate dataset

In [50]:
# Read pp rasters
pp_docs = os.listdir(r"C:\Users\pauls\Anaconda3\pkgs\sphy-2.2.1-py_0\site-packages\SPHY\Juncal_norte\input\climate_forcing\pp")
pp_docs

#Generate empty list
pp_dataset = []

for pp in pp_docs:
    name = r"C:\Users\pauls\Anaconda3\pkgs\sphy-2.2.1-py_0\site-packages\SPHY\Juncal_norte\input\climate_forcing\pp\pp"+pp[2:]
    pp_dataset.append(rio.open(name).read(1)[x,y])

pp_dataset = pd.DataFrame(pp_dataset)
pp_dataset.to_excel(r"C:\Users\pauls\Desktop\Proyectos\Juncal_norte\Climate_forcing\pp.xlsx")

In [5]:
pp = rio.open(r"G:\popoes\Projects\Juncal_norte\Climate_forcing\CR2MET_PR_JUNCALNORTE_500m.nc")
pp_cdf = xr.open_dataset(r"G:\popoes\Projects\Juncal_norte\Climate_forcing\CR2MET_PR_JUNCALNORTE_500m.nc")


pp_new_dataset = []
for i in range(len(pp_cdf["pr"])):
    ii = i+1
    pp_it = pp.read(ii)
    pp_new_dataset.append(pp_it[x,y])
    
pp_new_dataset = pd.DataFrame(pp_new_dataset)
pp_new_dataset.to_excel(r"C:\Users\pauls\Desktop\Proyectos\Juncal_norte\Climate_forcing\pp_fromnc.xlsx")

#### Generate new pp

In [47]:
#Read PP
pp = rio.open(r"G:\popoes\Projects\Juncal_norte\Climate_forcing\CR2MET_PR_JUNCALNORTE_500m.nc")
pp_cdf = xr.open_dataset(r"G:\popoes\Projects\Juncal_norte\Climate_forcing\CR2MET_PR_JUNCALNORTE_500m.nc") 

#Read net dem
netdem_t = rio.open(r"C:\Users\pauls\Desktop\Proyectos\Juncal_norte\GIS\net_dem.tif")
netdem = netdem_t.read(1)

#Read mask
mask_tif = rio.open(r"C:\Users\pauls\Desktop\Proyectos\Juncal_norte\GIS\juncalnorte_mask_v0.tif") #windows
mask_array = mask_tif.read(1)

b = 1.67 #adjustment factor reanalisis product and observed data
f1 = 2.4 #El Yeso - CR2 adjustment % realted to pp_avg
g_pp = 0.0084 #PP gradient mm/m related to pp_avg


for i in range(len(pp_cdf["pr"])):
    ii = i+1
    if ii < 10000:
        value = ii/1000
        f_value = "{:.3f}".format(value)
        #name = "/home/phi/Desktop/Projects/Juncal_norte/Climate_forcing/pp/pp00000"+str(f_value) #linux
        name = r"C:\Users\pauls\Desktop\Proyectos\Juncal_norte\Climate_forcing\pp2\pp20000"+str(f_value) #windows
    else:
        value = ii/1000
        f_value = "{:.3f}".format(value)
        #name = "/home/phi/Desktop/Projects/Juncal_norte/Climate_forcing/pp/pp0000"+str(f_value) #linux
        name = r"C:\Users\pauls\Desktop\Proyectos\Juncal_norte\Climate_forcing\pp2\pp2000"+str(f_value) #windows
    
    pp_it = pp.read(ii)
    if pp_it[36,21] == 0:
        pp_it = pp_it*0
    else:
        pp_it =  pp_it*0+pp_it[36,21]*b+g_pp*netdem-f1
        pp_it[pp_it <= 0] = 0
    print(name)
    maps = rio.open(
    str(name),"w",
    driver = "PCRaster",
    pcraster_valuescale = "VS_SCALAR",
    height = mask_tif.shape[0],
    width = mask_tif.shape[1],
    count = 1,
    nodata = mask_tif.nodata,
    dtype = np.float32,
    crs = 32719,
    transform = mask_tif.transform)
    maps.write(pp_it,1)
    maps.close()

C:\Users\pauls\Desktop\Proyectos\Juncal_norte\Climate_forcing\pp2\pp200000.001
C:\Users\pauls\Desktop\Proyectos\Juncal_norte\Climate_forcing\pp2\pp200000.002
C:\Users\pauls\Desktop\Proyectos\Juncal_norte\Climate_forcing\pp2\pp200000.003
C:\Users\pauls\Desktop\Proyectos\Juncal_norte\Climate_forcing\pp2\pp200000.004
C:\Users\pauls\Desktop\Proyectos\Juncal_norte\Climate_forcing\pp2\pp200000.005
C:\Users\pauls\Desktop\Proyectos\Juncal_norte\Climate_forcing\pp2\pp200000.006
C:\Users\pauls\Desktop\Proyectos\Juncal_norte\Climate_forcing\pp2\pp200000.007
C:\Users\pauls\Desktop\Proyectos\Juncal_norte\Climate_forcing\pp2\pp200000.008
C:\Users\pauls\Desktop\Proyectos\Juncal_norte\Climate_forcing\pp2\pp200000.009
C:\Users\pauls\Desktop\Proyectos\Juncal_norte\Climate_forcing\pp2\pp200000.010
C:\Users\pauls\Desktop\Proyectos\Juncal_norte\Climate_forcing\pp2\pp200000.011
C:\Users\pauls\Desktop\Proyectos\Juncal_norte\Climate_forcing\pp2\pp200000.012
C:\Users\pauls\Desktop\Proyectos\Juncal_norte\Climat

### Temperature

#### Create dataset

In [50]:
# Read tmax and tmin rasters
tmax_docs = os.listdir(r"C:\Users\pauls\Anaconda3\pkgs\sphy-2.2.1-py_0\site-packages\SPHY\Juncal_norte\input\climate_forcing\tmax")
tmin_docs = os.listdir(r"C:\Users\pauls\Anaconda3\pkgs\sphy-2.2.1-py_0\site-packages\SPHY\Juncal_norte\input\climate_forcing\tmin")
tavg_docs = os.listdir(r"C:\Users\pauls\Anaconda3\pkgs\sphy-2.2.1-py_0\site-packages\SPHY\Juncal_norte\input\climate_forcing\tavg")
tmax_docs
tmin_docs
tavg_docs

#Generate empty list
tmax_dataset = []
tmin_dataset = []
tavg_dataset = []

for tmax in tmax_docs:
    name = r"C:\Users\pauls\Anaconda3\pkgs\sphy-2.2.1-py_0\site-packages\SPHY\Juncal_norte\input\climate_forcing\tmax\tmax"+tmax[4:]
    tmax_dataset.append(rio.open(name).read(1)[x,y])

for tmin in tmin_docs:
    name = r"C:\Users\pauls\Anaconda3\pkgs\sphy-2.2.1-py_0\site-packages\SPHY\Juncal_norte\input\climate_forcing\tmin\tmin"+tmin[4:]
    tmin_dataset.append(rio.open(name).read(1)[x,y])

for tavg in tavg_docs:
    name = r"C:\Users\pauls\Anaconda3\pkgs\sphy-2.2.1-py_0\site-packages\SPHY\Juncal_norte\input\climate_forcing\tavg\tavg"+tavg[4:]
    tavg_dataset.append(rio.open(name).read(1)[x,y])

    
tmax_dataset = pd.DataFrame(tmax_dataset)
tmax_dataset.to_excel(r"C:\Users\pauls\Desktop\Proyectos\Juncal_norte\Climate_forcing\tmax.xlsx")

tmin_dataset = pd.DataFrame(tmin_dataset)
tmin_dataset.to_excel(r"C:\Users\pauls\Desktop\Proyectos\Juncal_norte\Climate_forcing\tmin.xlsx")

tavg_dataset = pd.DataFrame(tavg_dataset)
tavg_dataset.to_excel(r"C:\Users\pauls\Desktop\Proyectos\Juncal_norte\Climate_forcing\tavg.xlsx")

#### Generate temperature dataset

In [3]:
tmax_cdf = xr.open_dataset(r"G:\popoes\Projects\Juncal_norte\Climate_forcing\CR2MET_TMAX_JUNCALNORTE_500m.nc")

tmax = rio.open(r"G:\popoes\Projects\Juncal_norte\Climate_forcing\CR2MET_TMAX_JUNCALNORTE_500m.nc")
tmin = rio.open(r"G:\popoes\Projects\Juncal_norte\Climate_forcing\CR2MET_TMIN_JUNCALNORTE_500m.nc")

#Read net dem
netdem_t = rio.open(r"C:\Users\pauls\Desktop\Proyectos\Juncal_norte\GIS\net_dem.tif")
netdem = netdem_t.read(1)

#Read mask
mask_tif = rio.open(r"C:\Users\pauls\Desktop\Proyectos\Juncal_norte\GIS\juncalnorte_mask_v0.tif") #windows
mask_array = mask_tif.read(1)

#b1 = 1 #adjustment factor reanalisis product and observed data
#b2 = 1 #adjustment factor reanalisis product and observed data
#f1 = -1.92 #El Yeso - CR2 adjustment °C 
g_tmp = -0.0067 #PP gradient mm/m related to pp_avg

for i in range(len(tmax_cdf["tmax"])):
    ii = i+1
    if ii < 10000:
        value = ii/1000
        f_value = "{:.3f}".format(value)
        name = r"C:\Users\pauls\Desktop\Proyectos\Juncal_norte\Climate_forcing\tmax3\tmax300"+str(f_value)
        name2 = r"C:\Users\pauls\Desktop\Proyectos\Juncal_norte\Climate_forcing\tmin3\tmin300"+str(f_value)
        name3 = r"C:\Users\pauls\Desktop\Proyectos\Juncal_norte\Climate_forcing\tavg3\tavg300"+str(f_value)
    else:
        value = ii/1000
        f_value = "{:.3f}".format(value)
        name = r"C:\Users\pauls\Desktop\Proyectos\Juncal_norte\Climate_forcing\tmax3\tmax30"+str(f_value)
        name2 = r"C:\Users\pauls\Desktop\Proyectos\Juncal_norte\Climate_forcing\tmin3\tmin30"+str(f_value)
        name3 = r"C:\Users\pauls\Desktop\Proyectos\Juncal_norte\Climate_forcing\tavg3\tavg30"+str(f_value)
        
    tmax_it = tmax.read(ii)
    tmin_it = tmin.read(ii)
    
    #Elevation adjustment
    tmax_it = tmax_it*0+tmax_it[36,21]+g_tmp*netdem
    tmin_it = tmin_it*0+tmin_it[36,21]+g_tmp*netdem
    tavg_it = (tmax_it+tmin_it)/2
        
    #Tmax
    maps = rio.open(
    str(name),"w",
    driver = "PCRaster",
    pcraster_valuescale = "VS_SCALAR",
    height = mask_tif.shape[0],
    width = mask_tif.shape[1],
    count = 1,
    nodata = mask_tif.nodata,
    dtype = np.float32,
    crs = 32719,
    transform = mask_tif.transform)
    maps.write(tmax_it,1)
    maps.close()
    
    #Tmin
    maps2 = rio.open(
    str(name2),"w",
    driver = "PCRaster",
    pcraster_valuescale = "VS_SCALAR",
    height = mask_tif.shape[0],
    width = mask_tif.shape[1],
    count = 1,
    nodata = mask_tif.nodata,
    dtype = np.float32,
    crs = 32719,
    transform = mask_tif.transform)
    maps2.write(tmin_it,1)
    maps2.close()
    
    #Tavg
    maps3 = rio.open(
    str(name3),"w",
    driver = "PCRaster",
    pcraster_valuescale = "VS_SCALAR",
    height = mask_tif.shape[0],
    width = mask_tif.shape[1],
    count = 1,
    nodata = mask_tif.nodata,
    dtype = np.float32,
    crs = 32719,
    transform = mask_tif.transform)
    maps3.write(tavg_it,1)
    maps3.close()

    print(name)
    print(name2)
    print(name3)

C:\Users\pauls\Desktop\Proyectos\Juncal_norte\Climate_forcing\tmax3\tmax3000.001
C:\Users\pauls\Desktop\Proyectos\Juncal_norte\Climate_forcing\tmin3\tmin3000.001
C:\Users\pauls\Desktop\Proyectos\Juncal_norte\Climate_forcing\tavg3\tavg3000.001
C:\Users\pauls\Desktop\Proyectos\Juncal_norte\Climate_forcing\tmax3\tmax3000.002
C:\Users\pauls\Desktop\Proyectos\Juncal_norte\Climate_forcing\tmin3\tmin3000.002
C:\Users\pauls\Desktop\Proyectos\Juncal_norte\Climate_forcing\tavg3\tavg3000.002
C:\Users\pauls\Desktop\Proyectos\Juncal_norte\Climate_forcing\tmax3\tmax3000.003
C:\Users\pauls\Desktop\Proyectos\Juncal_norte\Climate_forcing\tmin3\tmin3000.003
C:\Users\pauls\Desktop\Proyectos\Juncal_norte\Climate_forcing\tavg3\tavg3000.003
C:\Users\pauls\Desktop\Proyectos\Juncal_norte\Climate_forcing\tmax3\tmax3000.004
C:\Users\pauls\Desktop\Proyectos\Juncal_norte\Climate_forcing\tmin3\tmin3000.004
C:\Users\pauls\Desktop\Proyectos\Juncal_norte\Climate_forcing\tavg3\tavg3000.004
C:\Users\pauls\Desktop\Proye